In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import requests
import json
import base64

from spotify_creds import *

%matplotlib inline

In [14]:
credentials = f'{client_id}:{client_secret}'
credbytes = credentials.encode('ascii')
credb64 = base64.b64encode(credbytes)
client_data = credb64.decode('ascii')

url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization': 'Basic ' + client_data}
data = {'grant_type': 'client_credentials'}

In [15]:
results = requests.post(url, headers=headers, data=data)

In [17]:
token = results.json()['access_token']
c_headers = {'Authorization': f"Bearer {token}"}

In [71]:
artist_1 = '196lKsA13K3keVXMDFK66q'

artist = requests.get(f'https://api.spotify.com/v1/artists/{artist_1}', headers=c_headers).json()
albums = requests.get(f'https://api.spotify.com/v1/artists/{artist_1}/albums', headers=c_headers).json()
# top_tracks = requests.get(f'https://api.spotify.com/v1/artists/{artist_1}/top-tracks/', headers=c_headers).json()
related_artists = requests.get(f'https://api.spotify.com/v1/artists/{artist_1}/related-artists', headers=c_headers).json()

In [86]:
related_artists['artists'][0].keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])